In [494]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
train.head()

,outlet_no,business_type,city,state,zip,store_location,avg_age,blue_collar,white_collar,female,...,total_household_income,time_zone,latitude,longitude,location_employee_code,employee_size,credit_score,credit_score_range,actual_credit_score,total_sales
0,247,1,LIMA,OH,45804,TRAVEL/TRANSPORTATION/HOSPITALITY,44.01,43.68,35.41,53.37,...,63860000,EST,40.704283,-84.147047,E,70,B,80 TO 84,80,4070
1,253,4,VACAVILLE,CA,95687,AT-WORK,38.36,41.31,26.48,54.00,...,92764000,PST,38.354860,-121.950360,D,35,A,90 TO 94,91,4323
2,265,4,CINCINNATI,OH,45219,EDUCATIONAL,28.81,17.22,43.06,45.56,...,51027000,EST,39.136100,-84.504288,C,13,A+,95 TO 100,97,2163
3,267,1,GLASGOW,KY,42141,AT-WORK,42.89,38.49,44.00,52.41,...,47119000,CST,37.010838,-85.903418,A,3,B+,85 TO 89,86,528
4,276,1,BOWLING GREEN,KY,42101,EATING & DRINKING,38.82,60.82,26.24,54.18,...,47941000,CST,36.964703,-86.436535,A,3,P,PROFESSIONAL INDIVIDUAL,78,528


In [499]:

train['zip1']=0
train['zip2']=0

test['zip1']=0
test['zip2']=0
for i in range(len(train)):
    p=str(train['zip'][i])
    train['zip1'][i]=int(p[:2])
    train['zip2'][i]=int(p[2:])
    
for j in range(len(test)):
    q=str(test['zip'][j])
    test['zip1'][j]=int(q[:2])
    test['zip2'][j]=int(q[2:])
    
train=train.drop('zip',1)
test=test.drop('zip',1)

In [501]:
cat_columns=['city', 'state', 'store_location', 'time_zone',
       'location_employee_code', 'credit_score']


In [502]:
train['credit_score_range_max']=0
train['credit_score_range_min']=0


test['credit_score_range_max']=0
test['credit_score_range_min']=0


for i in range(len(train)):
    try:
        train['credit_score_range_max'][i]=int(train.credit_score_range[i].split('TO')[1])
        train['credit_score_range_min'][i]=int(train.credit_score_range[i].split('TO')[0])
        train['credit_score_range'][i]=int(int(train.credit_score_range[i].split('TO')[1])-int(train.credit_score_range[i].split('TO')[0]))
    except:
        train['credit_score_range'][i]=0
        train['credit_score_range_max'][i]=0
        train['credit_score_range_min'][i]=0
    
    
for i in range(len(test)):
    try:
        test['credit_score_range_max'][i]=int(test.credit_score_range[i].split('TO')[1])
        test['credit_score_range_min'][i]=int(test.credit_score_range[i].split('TO')[0])
        test['credit_score_range'][i]=int(int(test.credit_score_range[i].split('TO')[1])-int(test.credit_score_range[i].split('TO')[0]))
    except:
        test['credit_score_range'][i]=0
        test['credit_score_range_max'][i]=0
        test['credit_score_range_min'][i]=0
    

In [503]:
for var in cat_columns:
                lb = LabelEncoder()
                full_var_data = pd.concat((train[var],test[var]),axis=0).astype('str')
                temp = lb.fit_transform(np.array(full_var_data))
                train[var] = lb.transform(np.array( train[var] ).astype('str'))
                test[var] = lb.transform(np.array( test[var] ).astype('str'))


In [504]:
train['credit_score_range'] = train['credit_score_range'].apply(pd.to_numeric)
test['credit_score_range'] = test['credit_score_range'].apply(pd.to_numeric)

In [505]:
def getCountVar(compute_df, count_df, var_name):
        grouped_df = count_df.groupby(var_name)
        count_dict = {}
        for name, group in grouped_df:
                count_dict[name] = group.shape[0]

        count_list = []
        for index, row in compute_df.iterrows():
                name = row[var_name]
                count_list.append(count_dict.get(name, 0))
        return count_list

In [506]:
#store_location
train['store_location_Count']=getCountVar(train,train,'store_location')
test['store_location_Count']=getCountVar(test,train,'store_location')
#time_zone
train['time_zone_Count']=getCountVar(train,train,'time_zone')
test['time_zone_Count']=getCountVar(test,train,'time_zone')
#location_employee_code
train['location_employee_code_Count']=getCountVar(train,train,'location_employee_code')
test['location_employee_code_Count']=getCountVar(test,train,'location_employee_code')


In [507]:
train['total_household_size ']=train['total_household_size ']
train['total_household_income_normalized']=train['total_household_income']/train['total_household_size ']
train=train.drop('total_household_income',1)

test['total_household_size ']=test['total_household_size ']
test['total_household_income_normalized']=test['total_household_income']/test['total_household_size ']
test=test.drop('total_household_income',1)

In [508]:
y=train.total_sales
train=train.drop(['total_sales','outlet_no'],1)
outlet=test.outlet_no
test=test.drop('outlet_no',1)

In [509]:
from xgboost import XGBRegressor
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV

In [ ]:
df=train.copy()
df=df.replace(np.inf, np.nan)
df=df.fillna(0)
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures


In [ ]:
for i in range(5,train.shape[1]):
    print(i)
    pca = PCA(n_components=i)
    mat=pca.fit_transform(df)
    model = XGBRegressor()
    learning_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
    n_estimators=[100,150,300,400,500]
    param_grid = dict(learning_rate=learning_rate,n_estimators=n_estimators)
    kfold = StratifiedKFold(y, n_folds=3, shuffle=True, random_state=7)
    grid_search = GridSearchCV(model, param_grid, scoring="mean_absolute_error", n_jobs=-1, cv=kfold)
    result = grid_search.fit(mat,y)
    print("Best: %f using %s" % (result.best_score_, result.best_params_))

5
Best: -9209.305989 using {'n_estimators': 100, 'learning_rate': 0.1}
6
Best: -9287.470591 using {'n_estimators': 100, 'learning_rate': 0.1}
7
Best: -9196.405943 using {'n_estimators': 100, 'learning_rate': 0.1}
8
Best: -9035.259406 using {'n_estimators': 100, 'learning_rate': 0.2}
9
Best: -9270.256172 using {'n_estimators': 100, 'learning_rate': 0.1}
10
Best: -9215.042149 using {'n_estimators': 150, 'learning_rate': 0.1}
11
Best: -9110.424646 using {'n_estimators': 150, 'learning_rate': 0.1}
12
Best: -9089.865888 using {'n_estimators': 150, 'learning_rate': 0.1}
13
Best: -9096.707098 using {'n_estimators': 150, 'learning_rate': 0.1}
14
Best: -9083.972788 using {'n_estimators': 100, 'learning_rate': 0.1}
15
Best: -9074.821866 using {'n_estimators': 150, 'learning_rate': 0.1}
16
Best: -9061.056211 using {'n_estimators': 150, 'learning_rate': 0.1}
17
Best: -9113.136108 using {'n_estimators': 150, 'learning_rate': 0.1}
18


In [ ]:
result = grid_search.fit(mat,y)
# summarize results

# Best: -8796.946124 using {'n_estimators': 150, 'learning_rate': 0.1}  n=7

print("Best: %f using %s" % (result.best_score_, result.best_params_))

In [ ]:
from sklearn import cross_validation
from sklearn.cross_validation import cross_val_score
model=XGBRegressor(learning_rate=result.best_params_['learning_rate'],n_estimators=result.best_params_['n_estimators'])
cv = StratifiedKFold(y, n_folds=3, shuffle=True, random_state=7)
print("Training Model")
model.fit(mat,y)


In [ ]:
y_pred1=model.predict(mat_test)

In [ ]:
output1 = pd.DataFrame( data={"outlet_no":outlet,"total_sales_Actual": y_pred1} )
output1.to_csv("pca.csv", index=False,quoting=3)

In [492]:
train.head()

,outlet_no,business_type,city,state,store_location,avg_age,blue_collar,white_collar,female,male,...,latitude,longitude,location_employee_code,employee_size,credit_score,credit_score_range,actual_credit_score,total_sales,zip1,zip2
0,247,1,LIMA,OH,TRAVEL/TRANSPORTATION/HOSPITALITY,44.01,43.68,35.41,53.37,46.63,...,40.704283,-84.147047,E,70,B,80 TO 84,80,4070,14,75
1,253,4,VACAVILLE,CA,AT-WORK,38.36,41.31,26.48,54.00,46.00,...,38.354860,-121.950360,D,35,A,90 TO 94,91,4323,14,75
2,265,4,CINCINNATI,OH,EDUCATIONAL,28.81,17.22,43.06,45.56,54.44,...,39.136100,-84.504288,C,13,A+,95 TO 100,97,2163,14,75
3,267,1,GLASGOW,KY,AT-WORK,42.89,38.49,44.00,52.41,47.59,...,37.010838,-85.903418,A,3,B+,85 TO 89,86,528,14,75
4,276,1,BOWLING GREEN,KY,EATING & DRINKING,38.82,60.82,26.24,54.18,45.82,...,36.964703,-86.436535,A,3,P,PROFESSIONAL INDIVIDUAL,78,528,14,75


In [500]:
train.head()

,outlet_no,business_type,city,state,store_location,avg_age,blue_collar,white_collar,female,male,...,latitude,longitude,location_employee_code,employee_size,credit_score,credit_score_range,actual_credit_score,total_sales,zip1,zip2
0,247,1,LIMA,OH,TRAVEL/TRANSPORTATION/HOSPITALITY,44.01,43.68,35.41,53.37,46.63,...,40.704283,-84.147047,E,70,B,80 TO 84,80,4070,45,804
1,253,4,VACAVILLE,CA,AT-WORK,38.36,41.31,26.48,54.00,46.00,...,38.354860,-121.950360,D,35,A,90 TO 94,91,4323,95,687
2,265,4,CINCINNATI,OH,EDUCATIONAL,28.81,17.22,43.06,45.56,54.44,...,39.136100,-84.504288,C,13,A+,95 TO 100,97,2163,45,219
3,267,1,GLASGOW,KY,AT-WORK,42.89,38.49,44.00,52.41,47.59,...,37.010838,-85.903418,A,3,B+,85 TO 89,86,528,42,141
4,276,1,BOWLING GREEN,KY,EATING & DRINKING,38.82,60.82,26.24,54.18,45.82,...,36.964703,-86.436535,A,3,P,PROFESSIONAL INDIVIDUAL,78,528,42,101
